# Data Exploaraion

In [ ]:
import os

In [ ]:
data_path = "/kaggle/input/camvid/CamVid"

In [ ]:
print('Number of train frames: ' + str(len(os.listdir(data_path + '/'+ 'train'))))
print('Number of train labels: ' + str(len(os.listdir(data_path + '/'+ 'train_labels'))))
print('Number of val frames: ' + str(len(os.listdir(data_path + '/'+ 'val'))))
print('Number of val labels: ' + str(len(os.listdir(data_path + '/'+ 'val_labels'))))
print('Number of test frames: ' + str(len(os.listdir(data_path + '/'+ 'test'))))
print('Number of test labels: ' + str(len(os.listdir(data_path + '/'+ 'test_labels'))))
print('Total frames: ' + str(len(os.listdir(data_path + '/'+ 'train')) + len(os.listdir(data_path + '/'+ 'val')) + len(os.listdir(data_path + '/'+ 'test'))))

Now, let's see which classes we have. This can be found in the original CAMVID [text file](http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/data/label_colors.txt). However, under the same repo, the author has dumped it into csv which we will use.

In [ ]:
import pandas as pd
classes = pd.read_csv(data_path+'/class_dict.csv', index_col =0)

In [ ]:
classes

In [ ]:
n_classes = len(classes)
n_classes

**This data frame maps the class names to colors.**

**To access the colors, we can index the dataframe with its row index name using the .loc operation.**


Now we are ready to create a map from class name to color

In [ ]:
cls2rgb = {cl:list(classes.loc[cl, :]) for cl in classes.index}

In [ ]:
cls2rgb

## Now let's visualize and explore some samples:

In [ ]:
from glob import glob
import numpy as np
train_img_loc = sorted(np.array(glob("/kaggle/input/camvid/CamVid/train/*")))
train_labels_loc = sorted(np.array(glob("/kaggle/input/camvid/CamVid/train_labels/*")))

In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt


idx = 0
img = cv2.imread(train_img_loc[idx])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(img.shape)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
img_arr = img.reshape(-1, 3) # combining the width and height and keeping the channels
kmeans = KMeans(n_clusters=5)
kmeans.fit(img_arr)

In [ ]:
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(img.shape)

In [ ]:
plt.imshow(segmented_img / 255)

Let's have a look on the masks (the ground truth)

As you can see the masks are just colors (L,W,3).
What we actually want is a (L,W) matrix, with each value is from 0 to 31 representing the 32 class labels.

Colors are different from the colors in cls2rgb! Because the order is BGR not RGB when using cv2.imread: https://stackoverflow.com/questions/46898979/how-to-check-the-channel-order-of-an-image

If you want to get the same order as in the color mapping of CAMVID, use the cv converted

In [ ]:
import numpy as np

idx = 0
mask = cv2.imread(train_labels_loc[idx])
mask = cv2.cvtColor((mask).astype(np.uint8), cv2.COLOR_BGR2RGB)# If you want to get the same order as in the color mapping of CAMVID, use the cv converted

Now if you plot the mask again, you will see different colors. For example the red and blue are reversed than before:

In [ ]:
plt.imshow(mask)
print(mask.shape)

Another solution is to use load_image from keras which uses RGB (it uses PIL under the hood) unlike cv2.imread

In [ ]:
from keras.preprocessing.image import load_img
# mask = load_img(str(data_path) + '/train_labels/0001TP_006690_L.png')
mask = load_img(train_labels_loc[0])
mask


In [ ]:
mask = np.array(mask)# Now colors are the same as in the dict, since keras load_img uses RGB order.

In [ ]:
mask.shape

In [ ]:
np.unique(mask)

In [ ]:
mask.shape

In [ ]:
plt.imshow(np.where([[64, 64, 0]], mask, 0))

In [ ]:
idx = 0
mask = cv2.imread(train_labels_loc[idx])
mask = cv2.cvtColor((mask).astype(np.uint8), cv2.COLOR_BGR2RGB)# If you want to get the same order as in the color mapping of CAMVID, use the cv converted

In [ ]:
class_clr = [ color for color in list( cls2rgb.values() ) ]

class_name = list( cls2rgb.keys() )

# class_id = 1
for class_id in range(32):
    true_case = np.float32( np.all( np.equal( class_clr[ class_id ], mask ), axis=-1 ) * 1 )

    if true_case.any():
        contours, _ = cv2.findContours(true_case.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # Get the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Draw the rectangle on the mask
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(img, class_name[class_id], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    else:
        continue

plt.imshow(img)

In [ ]:
idx = 0
img = cv2.imread(train_img_loc[idx])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.imshow(img)
# print(img.shape)

pedestrian = 26
true_case = np.float32( np.all( np.equal( class_clr[ pedestrian ], mask ), axis=-1 ) * 1 )

if true_case.any():
    contours, _ = cv2.findContours(true_case.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Get the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Draw the rectangle on the mask
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(img, class_name[pedestrian], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(30, 10))

plt.imshow(img)

In [ ]:
c_st =  []
for colour in list( cls2rgb.values() ):   
    c_st.append( colour )
    
class_name = list( (cls2rgb.keys()) ) 
class_id = 26

case_all_class = np.float32( np.equal( c_st[ class_id ], mask ) * 1)
case_true_class =  np.float32(np.all(np.equal( c_st[ class_id ], mask), axis = -1 ) *1 )


figsize=(15, 3)
_, axes = plt.subplots( nrows=1, ncols= 3, figsize=figsize )

axes[0].imshow( mask )
axes[1].imshow( case_all_class )
axes[2].imshow( case_true_class, cmap='gray' )

print('name of the class:', class_name[ class_id ] )

# plt.imshow(case_all_class)
# plt.imshow(case_true_class)

In [ ]:
def adjust_mask(mask, flat=False):
    
    semantic_map = []
    for colour in list(cls2rgb.values()):        
        equality = np.equal(mask, colour)# 256x256x3 with True or False
        class_map = np.all(equality, axis = -1)# 256x256 If all True, then True, else False
        semantic_map.append(class_map)# List of 256x256 arrays, map of True for a given found color at the pixel, and False otherwise.
    semantic_map = np.stack(semantic_map, axis=-1)# 256x256x32 True only at the found color, and all False otherwise.
    if flat:
        semantic_map = np.reshape(semantic_map, (-1,256*256))

    return np.float32(semantic_map)# convert to numbers

In [ ]:
new_mask = adjust_mask(mask)

In [ ]:
new_mask.shape

In [ ]:
idx2rgb={idx:np.array(rgb) for idx, (cl, rgb) in enumerate(cls2rgb.items())}

In [ ]:
# idx2rgb

In [ ]:
# Map the idx back to rgb

def map_class_to_rgb(p):
  
  return idx2rgb[p[0]]

rgb_mask = np.apply_along_axis(map_class_to_rgb, -1, np.expand_dims(np.argmax(new_mask, axis=-1), -1))

In [ ]:
plt.imshow(rgb_mask)

In [ ]:

model = unet(n_classes)
model.summary()

In [ ]:
import numpy as np 
import os
import tensorflow as tf
#import skimage.io as io
#import skimage.transform as trans
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import *
from tensorflow.keras.regularizers import * 
from keras.models import Model
from keras.layers import Input
from keras.preprocessing.image import load_img
from tensorflow.keras  import backend as keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Res_PSP_V1.3 with Augmentation

## Residual_block

In [ ]:
def res_block(in_tensor, n_filter, kernel_init, do_norm=True):

    # in_tensor --> Incoming input
    # n_filter  --> Number of filters
    # s_kernel  --> Kernel size
    # do_norm   --> Use normalization or not

    in_tensor_0 = Conv2D(n_filter , kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init)(in_tensor)

    # ftr1 = Conv2D(n_filter , kernel_size=(7,7), padding='same', activation='relu', kernel_initializer=kernel_init)(in_tensor)
    ftr1 = Conv2D(n_filter , kernel_size=(3,3), dilation_rate=(5, 5), padding='same', activation='relu', kernel_initializer=kernel_init)(in_tensor)
    ftr1 = Conv2D(n_filter , kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr1)
    ftr1 = DepthwiseConv2D(kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr1)
    ftr1 = DepthwiseConv2D(kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr1)
    ftr1 = DepthwiseConv2D(kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr1)
    ftr1 = add([in_tensor_0, ftr1])
    ftr1 = BatchNormalization()(ftr1)
    drop1 = Dropout(0.2)(ftr1)
    pool1 = MaxPooling2D(2)(drop1)

    # ftr2_res = Conv2D(n_filter * 2 , kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=kernel_init)(pool1)
    ftr2_res = Conv2D(n_filter * 2 , kernel_size=(3, 3), dilation_rate=(3, 3), padding='same', activation='relu', kernel_initializer=kernel_init)(pool1)
    ftr2 = Conv2D(n_filter * 2 , kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr2_res)
    ftr2 = DepthwiseConv2D(kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr2)
    ftr2 = DepthwiseConv2D(kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr2)
    ftr2 = DepthwiseConv2D(kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr2)
    ftr2 = add([ftr2_res, ftr2])
    ftr2 = BatchNormalization()(ftr2)
    drop2 = Dropout(0.3)(ftr2)
    pool2 = MaxPooling2D(2)(drop2)

    # ftr3_res = Conv2D(n_filter * 4 , kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(pool2)
    ftr3_res = Conv2D(n_filter * 4 , kernel_size=(3, 3), dilation_rate=(2, 2), padding='same', activation='relu', kernel_initializer=kernel_init)(pool2)
    ftr3 = Conv2D(n_filter * 4 , kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr3_res)
    ftr3 = DepthwiseConv2D(kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr3)
    ftr3 = DepthwiseConv2D(kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr3)
    ftr3 = DepthwiseConv2D(kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr3)
    ftr3 = add([ftr3_res, ftr3])
    ftr3 = BatchNormalization()(ftr3)
    drop3 = Dropout(0.3)(ftr3)
    pool3 = MaxPooling2D(2)(drop3)

    up1 = UpSampling2D()(drop3)
    merge1 = concatenate([drop2, up1])

    merge1 = BatchNormalization()(merge1)

    # ftr4_res = Conv2D(n_filter * 4 , kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(merge1)
    ftr4_res = Conv2D(n_filter * 4 , kernel_size=(3, 3), dilation_rate=(2, 2), padding='same', activation='relu', kernel_initializer=kernel_init)(merge1)
    ftr4 = Conv2D(n_filter * 4 , kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr4_res)
    ftr4 = DepthwiseConv2D(kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr4)
    ftr4 = DepthwiseConv2D(kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr4)
    ftr4 = DepthwiseConv2D(kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr4)
    ftr4 = add([ftr4_res, ftr4]) # 128, 128, 256
    ftr4 = BatchNormalization()(ftr4)
    drop4 = Dropout(0.3)(ftr4)

    up2 = UpSampling2D()(drop4) # 256, 256, 256
    merge2 = concatenate([drop1, up2])

    merge2 = BatchNormalization()(merge2)

    ftr5_res = Conv2D(n_filter, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(merge2)
    ftr5 = Conv2D(n_filter, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr5_res)
    ftr5 = DepthwiseConv2D(kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr5)
    ftr5 = DepthwiseConv2D(kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr5)
    ftr5 = DepthwiseConv2D(kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=kernel_init)(ftr5)
    ftr5 = add([ftr5_res, ftr5]) # 128, 128, 256
    ftr5 = BatchNormalization()(ftr5)
    drop5 = Dropout(0.3)(ftr5)

    if do_norm:
        skip_conn = add([drop5, in_tensor_0])
        res_bn = BatchNormalization()(skip_conn)
        # ups = UpSampling2D()(pool1)

    return res_bn


In [ ]:
def base_feature_maps(input_layer):
    # base covolution module to get input image feature maps

    # # block_1
    # base = conv_block(input_layer,[32,32,64],'1')
    # # block_2
    # base = conv_block(base,[64,64,128],'2')
    # # block_3
    # base = conv_block(base,[128,128,256],'3')
    # return base

    # base = res_block(input_layer, n_filter=32, s_kernel=3)
    base = res_block(input_layer, n_filter=64, s_kernel=3)
    base = res_block(base, n_filter=128, s_kernel=5)
    base = res_block(base, n_filter=256, s_kernel=7)

    return base

def pyramid_feature_maps(input_layer):
    # pyramid pooling module

    # base = base_feature_maps(input_layer)
    base = res_block(input_layer, n_filter=64, kernel_init="he_normal")
    print(base.shape)
    # red
    red = GlobalAveragePooling2D(name='red_pool')(base)
    red = tf.keras.layers.Reshape((1,1,-1))(red)
    red = Conv2D(filters=64,kernel_size=(1,1),name='red_1_by_1', activation='relu', kernel_initializer="he_normal")(red)
#     red = BatchNormalization()(red)
    red = DepthwiseConv2D(kernel_size=(3,3), padding='same', activation='relu', kernel_initializer="he_normal")(red)
    red = DepthwiseConv2D(kernel_size=(5,5), padding='same', activation='relu', kernel_initializer="he_normal")(red)
#     red = BatchNormalization()(red)
    red = UpSampling2D(size=256,interpolation='bilinear',name='red_upsampling')(red)

#     red = BatchNormalization()(red)
    # red = Conv2D(filters=64,kernel_size=(3,3),name='red_1_by_1', kernel_initializer="he_normal")(red)

    # yellow
    yellow = AveragePooling2D(pool_size=(2,2),name='yellow_pool')(base)
    yellow = Conv2D(filters=128,kernel_size=(1,1),name='yellow_1_by_1', activation='relu', kernel_initializer="he_normal")(yellow)
#     yellow = BatchNormalization()(yellow)
    yellow = DepthwiseConv2D(kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer="he_normal")(yellow)
    yellow = DepthwiseConv2D(kernel_size=(5, 5), padding='same', activation='relu', kernel_initializer="he_normal")(yellow)
#     yellow = BatchNormalization()(yellow)
    yellow = UpSampling2D(size=2,interpolation='bilinear',name='yellow_upsampling')(yellow)

    # blue
    blue = AveragePooling2D(pool_size=(4,4),name='blue_pool')(base)
    blue = Conv2D(filters=256,kernel_size=(1,1),name='blue_1_by_1', activation='relu', kernel_initializer="he_normal")(blue)
#     blue = BatchNormalization()(blue)
    blue = DepthwiseConv2D(kernel_size=(3, 3), padding='same', activation='relu',  kernel_initializer="he_normal")(blue)
    blue = DepthwiseConv2D(kernel_size=(5, 5), padding='same', activation='relu', kernel_initializer="he_normal")(blue)
#     blue = BatchNormalization()(blue)
    blue = UpSampling2D(size=4,interpolation='bilinear',name='blue_upsampling')(blue)

    # green
    green = AveragePooling2D(pool_size=(8,8),name='green_pool')(base)
    green = Convolution2D(filters=128,kernel_size=(1,1),name='green_1_by_1', activation='relu', kernel_initializer="he_normal")(green)
#     green = BatchNormalization()(green)
    green = DepthwiseConv2D(kernel_size=(3, 3), padding='same', activation='relu', kernel_initializer="he_normal")(green)
    green = DepthwiseConv2D(kernel_size=(5, 5), padding='same', activation='relu', kernel_initializer="he_normal")(green)
    green = UpSampling2D(size=8,interpolation='bilinear',name='green_upsampling')(green)

    # base + red + yellow + blue + green

    return BatchNormalization()(concatenate([base,red,yellow,blue,green]))

def last_conv_module(input_layer, num_classes):
    X = pyramid_feature_maps(input_layer)
    X = Convolution2D(filters=64, kernel_size=3,padding='same', activation='relu', name='second_last_conv_3_by_3')(X)
    X = Convolution2D(filters=num_classes, kernel_size=3,padding='same',name='last_conv_3_by_3')(X)
    X = BatchNormalization(name='last_conv_3_by_3_batch_norm')(X)
    X = Activation('softmax' ,name='last_conv_softmax')(X)
    # X = Flatten(name='last_conv_flatten')(X)
    return X

input_size = (256, 256, 3)
n_classes = 32
input_layer = tf.keras.Input(shape=input_size, name='input')
output_layer = last_conv_module(input_layer, n_classes)
# model_3 = tf.keras.Model(inputs=input_layer, outputs=output_layer)
# model_4 = tf.keras.Model(inputs=input_layer, outputs=output_layer)
# model_5 = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model_6 = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model_3.summary()

In [ ]:
model_4.summary()

In [ ]:
model_5.summary()

In [ ]:
model_6.summary()

## Loading The data to memory

In [ ]:
def load_raw_CAMVID(data_type='train', enc='ohe', shape='normal'):
    
    img_path = str(data_path) + '/' + data_type + '/'
    labels_path = str(data_path) + '/' + data_type + '_labels/'
    
    # without adding target_size=(256,256) in load_img we get Out of mem: 421x960x720x32x4bytes is around 34GB!
    x = np.array([np.array(load_img(str(img_path) + file, target_size=(256,256)))*1./255 for file in sorted(os.listdir(img_path))])
    
    if(enc=='ohe'):
        y = np.array([np.array(load_img(str(labels_path) + file, target_size=(256,256))) for file in sorted(os.listdir(labels_path))])
        
    elif(enc=='sparse_cat'):
        y = np.array([np.array(load_img(str(labels_path) + file, target_size=(256,256))) for file in sorted(os.listdir(labels_path))])
        
    if(shape == 'flat'):
        y = np.reshape(y.shape[0], y.shape[1]*y.shape[2])
        y = np.expand_dims(y, axis=-1)
        
    return x, y

In [ ]:
import time
start = time.time()
x_train, y_train = load_raw_CAMVID(data_type='train')
#x_test, y_test = load_raw_CAMVID(data_type='test')# Don't load test for RAM consumption
x_val, y_val = load_raw_CAMVID(data_type='val')
end = time.time()
print('Time elapsed: ', end-start)

print(x_train.shape)
print(y_train.shape)
#print(x_test.shape)
#print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

In [ ]:
x_test, y_test = load_raw_CAMVID(data_type='test')

## Augmentaton Block

In [ ]:
# Data generator
batch_sz = 4
#https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator
# we create two instances with the same arguments

# VI Note: use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
                    #rescale=1./255)# Data is already scaled when loaded

mask_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
                    #preprocessing_function=adjust_mask)# This is not possible since the preprocessing_function can only return the same shape as image

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen  = ImageDataGenerator(**mask_gen_args) 

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
#image_datagen.fit(images, augment=True, seed=seed)
#mask_datagen.fit(masks, augment=True, seed=seed)

image_generator = image_datagen.flow(
    x_train,
    seed=seed,
    batch_size=batch_sz)

mask_generator = mask_datagen.flow( 
    y_train,
    seed=seed,
    batch_size=batch_sz)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

def train_generator_fn():

    for (img, mask) in train_generator:
        new_mask = adjust_mask(mask)
        yield (img, new_mask)  
        
val_image_generator = image_datagen.flow(
    x_val,
    seed=seed,
    batch_size=batch_sz)

val_mask_generator = mask_datagen.flow(
    y_val,
    seed=seed,
    batch_size=batch_sz)

# combine generators into one which yields image and masks
val_generator = zip(val_image_generator, val_mask_generator)        
        
def val_generator_fn():

    for (img, mask) in val_generator:
        new_mask = adjust_mask(mask)
        yield (img, new_mask)         


## Metrics and losses

In [ ]:
def train_generator_fn():
    for (img,mask) in train_generator:
    
        
        new_mask = adjust_mask(mask)
        yield (img,new_mask)   
        
def val_generator_fn():
    for (img,mask) in val_generator:
        new_mask = adjust_mask(mask)
        yield (img,new_mask)  

        
## Metrics

def dice(y_true, y_pred, smooth=1):
    
    intersection = K.sum(y_true * y_pred, axis=[-1])
    union = K.sum(y_true, axis=[-1]) + K.sum(y_pred, axis=[-1])
    dicef = K.mean((2. * intersection + smooth)/(union + smooth), axis=-1)
    return dicef


def IOU(y_true, y_pred, smooth=1):

    intersection = K.sum(y_true * y_pred, axis=[-1])
    union = K.sum(y_true, axis=[-1]) + K.sum(y_pred, axis=[-1])-intersection
    iou_scr = K.mean(( intersection + smooth)/(union + smooth), axis=-1)
    return iou_scr

def IOU_loss(y_true, y_pred, smooth=1):

    intersection = K.sum(y_true * y_pred, axis=[-1])
    union = K.sum(y_true, axis=[-1]) + K.sum(y_pred, axis=[-1])-intersection
    iou_scr = K.mean(( intersection + smooth)/(union + smooth), axis=-1)
    return 1 - iou_scr


alpha_vals = np.ones((1, n_classes))  * 0.25

def categorical_focal_loss(alpha, gamma=2.):
    """
    Softmax version of focal loss.
    When there is a skew between different categories/labels in your data set, you can try to apply this function as a
    loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1

      where m = number of classes, c = class and o = observation

    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy. Alpha is used to specify the weight of different
      categories/labels, the size of the array needs to be consistent with the number of classes.
      gamma -- focusing parameter for modulating factor (1-p)

    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper

    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy

    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=[[.25, .25, .25]], gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    alpha = np.array(alpha, dtype=np.float32)

    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(K.sum(loss, axis=-1))

    return categorical_focal_loss_fixed

## Helper Callbacks

In [ ]:
import keras.callbacks as callbacks
from keras.callbacks import Callback

class SnapshotCallbackBuilder:
    def __init__(self, nb_epochs, nb_snapshots, init_lr=0.1):
        self.T = nb_epochs
        self.M = nb_snapshots
        self.alpha_zero = init_lr

    def get_callbacks(self, model_prefix='Model'):

        callback_list = [
            callbacks.ModelCheckpoint("./pspnet_camvid_1.hdf5",monitor='val_loss', 
                                   mode = 'min', save_best_only=True, verbose=1),
#             swa,
            callbacks.LearningRateScheduler(schedule=self._cosine_anneal_schedule),
            callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, mode='min', verbose=1, factor=0.5) # used on model_4
#             callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, mode='min', verbose=1, factor=0.2) # used on model_5
        ]

        return callback_list

    def _cosine_anneal_schedule(self, t):
        cos_inner = np.pi * (t % (self.T // self.M))  # t - 1 is used when t has 1-based indexing.
        cos_inner /= self.T // self.M
        cos_out = np.cos(cos_inner) + 1
        return float(self.alpha_zero / 2 * cos_out)

In [ ]:
# Model 3 #

Adam =  tf.keras.optimizers.Adam
# model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy',dice, IOU])

# model_2.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy', dice, IOU])

# model_3.compile(optimizer='adam', loss=[categorical_focal_loss(alpha=alpha_vals, gamma=2.)], 
#                 metrics = ['accuracy', dice, IOU])

# model_4.compile(optimizer='adam', loss=[categorical_focal_loss(alpha=alpha_vals, gamma=2.)], 
#                 metrics = ['accuracy', dice, IOU])

# model_5.compile(optimizer='adam', loss=[categorical_focal_loss(alpha=alpha_vals, gamma=2.)], 
#                 metrics = ['accuracy', dice, IOU])


model_6.compile(optimizer='adam', loss = 'categorical_crossentropy', 
                 metrics = ['accuracy', dice, IOU])

In [ ]:
BASE_PATH = "/kaggle/input/camvid/CamVid"
view = 0
batch_sz = 4
epochs = 50
validation_steps = 32

n_train_samples = len(os.listdir(str(BASE_PATH) + '/train/'))
snapshot = SnapshotCallbackBuilder(nb_epochs=epochs, nb_snapshots=1, init_lr=1e-3)
# n_train_samples

# model_checkpoint = ModelCheckpoint('pspnet_camvid.hdf5', monitor='val_loss', verbose=1, save_best_only=True)


model_6.fit_generator(train_generator_fn(),
                    validation_data=val_generator_fn(),
                    steps_per_epoch=n_train_samples//batch_sz,
                    validation_steps=validation_steps,
                    epochs=epochs,
#                     callbacks=[model_checkpoint]
                    callbacks=snapshot.get_callbacks())

## Visualizing model performance

In [ ]:
def visualize_seg(img, gt_mask, get_model=None, shape='normal', gt_mode='sparse'):
    fig , ax = plt.subplots(1,3,figsize=(15,15))
  
  # Img
    ax[0].imshow(img)
    ax[0].set_title("Orignal Image")
  
  # Predict
    pred_mask = get_model.predict(np.expand_dims(img, 0))
    pred_mask = np.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[0]
    if shape=='flat':
        pred_mask = np.reshape(pred_mask, (256,256)) # Reshape only if you use the flat model. O.w. you dont need
  
    rgb_mask = np.apply_along_axis(map_class_to_rgb, -1, np.expand_dims(pred_mask, -1))
  
  # Prediction
    ax[1].imshow(rgb_mask)
    ax[1].set_title("Predicted Mask")

              
  # GT mask
    if gt_mode == 'ohe':
        gt_img_ohe = np.argmax(gt_mask, axis=-1)
        gt_mask = np.apply_along_axis(map_class_to_rgb, -1, np.expand_dims(gt_img_ohe, -1))              
  
    ax[2].imshow((gt_mask).astype(np.uint8))
    ax[2].set_title("Ground truth")

In [ ]:
img = next(val_image_generator)[0]
gt_img = next(val_mask_generator)[0]
# visualize_seg(img, gt_img, get_model=model_4, gt_mode='sparse')
visualize_seg(img, gt_img, get_model=model_6, gt_mode='sparse')

In [ ]:
img = x_val[1]
gt_img = y_val[1]
# visualize_seg(img, gt_img, get_model=model_4, gt_mode='sparse')
visualize_seg(img, gt_img, get_model=model_6, gt_mode='sparse')

## Computing class-wise perforamce

In [ ]:
def compute_metrics(y_true, y_pred):
    '''
    Computes IOU and Dice Score.

    Args:
    y_true (tensor) - ground truth label map
    y_pred (tensor) - predicted label map
    '''

    class_wise_iou = []
    class_wise_dice_score = []

    smoothening_factor = 0.00001

    for i in range(32):
        intersection = np.sum((y_pred == i) * (y_true == i))
        y_true_area = np.sum((y_true == i))
        y_pred_area = np.sum((y_pred == i))
        combined_area = y_true_area + y_pred_area

        iou = (intersection + smoothening_factor) / (combined_area - intersection + smoothening_factor)
        class_wise_iou.append(iou)

        dice_score =  2 * ((intersection + smoothening_factor) / (combined_area + smoothening_factor))
        class_wise_dice_score.append(dice_score)

    return class_wise_iou, class_wise_dice_score

In [ ]:
img = x_test[1]
gt_img = y_test[1]
model_name = model_6
p_img = model_name.predict(np.expand_dims(img, 0))[0]
pred_mask = np.argmax(p_img, axis=-1)
rgb_mask = np.apply_along_axis(map_class_to_rgb, -1, np.expand_dims(pred_mask, -1))
im_iou, im_dice = compute_metrics(np.uint8(gt_img),np.uint8(rgb_mask))

print('iou score, dice score', im_iou, im_dice)

In [ ]:
np.uint8(rgb_mask)